In [31]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import time 

Ant colony optimisation

In [32]:
def distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

def aco(points,ants,iters,alpha,beta,evapo_rate,Q):
    npoints=len(points)
    pheromone=np.ones((npoints,npoints))
    best_path=None
    best_path_length= np.inf
    
    for iteration in range(iters):
        paths=[]
        path_lengths=[]
        
        for ant in range(ants):
            visited=[False]*npoints
            curr_point=np.random.randint(npoints)
            visited[curr_point]=True
            path=[curr_point]
            path_length=0
            
            while False in visited:
                unvisit=np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisit))
               
                for i,unvis in enumerate(unvisit):
                      probabilities[i] = (pheromone[curr_point, unvis]**alpha)*((Q/distance(points[curr_point], points[unvis]))**beta)
                   
                
                probabilities/=np.sum(probabilities)
                
                next_point=np.random.choice(unvisit,p=probabilities)   
                path.append(next_point)
                path_length+=distance(points[curr_point], points[next_point])
                visited[next_point]=True
                curr_point=next_point
            paths.append(path)
            path_lengths.append(path_length)
            if path_length<best_path_length:
                best_path=path
                best_path_length=path_length
                
        pheromone*=evapo_rate
            
        for path,path_length in zip(paths,path_lengths):
                for i in range(npoints-1):
                    pheromone[path[i],path[i+1]]+=Q/path_length
                pheromone[path[-1], path[0]] += Q/path_length
  
    ind_map = plt.imread("india3.jpg")

    fig, ax = plt.subplots()
    ax.imshow(ind_map)
    ax.scatter(points[:, 0], points[:, 1], c='r', marker='o')

    
    for i in range(npoints-1):
        ax.plot([points[best_path[i],0], points[best_path[i+1],0]],
                [points[best_path[i],1], points[best_path[i+1],1]],
    
                c='g', linestyle='-', linewidth=2, marker='o')
        
    ax.plot([points[best_path[0],0], points[best_path[-1],0]],
            [points[best_path[0],1], points[best_path[-1],1]],

            c='g', linestyle='-', linewidth=2, marker='o')
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')

    plt.show()      
    plt.savefig("graph{}".format(ants))         
    print(path)    
    return best_path_length

%Collecting points

In [33]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

matplotlib.use('TkAgg')

data = []

def collect_points_on_image(image, n):
    global data
    plt.imshow(image)
    plt.title("Click to collect points")
    plt.ion()
    points = []

    def on_click(event):
        nonlocal points
        if event.inaxes:
            points.append((event.xdata, event.ydata))
            plt.scatter(event.xdata, event.ydata, color='red')
            if len(points) > 1:
                for i in range(len(points)-1):             
                    plt.plot([points[i][0], points[-1][0]], [points[i][1], points[-1][1]], color='red')
            plt.draw()

        if len(points) == n:
            plt.savefig("collected_points.png")
            plt.ioff()
            plt.show(block=False)
            plt.close()
            global data
            data = points  
            

    plt.gcf().canvas.mpl_connect('button_press_event', on_click)
    plt.show(block=False)

image_path = "india3.jpg"
ind_map = mpimg.imread(image_path)

   
   
collect_points_on_image(ind_map, 10)

In [34]:
data=np.array(data)
(aco(data,30,100,1,1,0.5,0.1))

[1, 0, 2, 3, 4, 5, 6, 7, 8, 9]


1121.5266741060227

% Ant colony optimisation with obstacles

In [35]:
def perpendicular_distance(point, line_start, line_end):
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end
    
    numerator = np.abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    
    distance = numerator / denominator
    return distance

def path_check(path, obstacles, distance_factor):
    for i in range(len(path) - 1):
        point1 = path[i]
        point2 = path[i + 1]
        
        for obstacle in obstacles:
            dist = perpendicular_distance(obstacle,point1, point2)
            if dist < distance_factor:
                return False
        if(perpendicular_distance(obstacle,path[0], path[len(points)-1])<distance_factor):
            return False      
    return True 
    



def distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

def acob(points,ants,iters,alpha,beta,evapo_rate,Q,obstacles,distance_factor):
    npoints=len(points)
    pheromone=np.ones((npoints,npoints))
    best_path=None
    best_path_length= np.inf
    all_paths = []
    for iteration in range(iters):
        paths=[]
        path_lengths=[]
        
        for ant in range(ants):
            visited=[False]*npoints
            curr_point=np.random.randint(npoints)
            visited[curr_point]=True
            path=[curr_point]
            path_length=0
            
            while False in visited:
                unvisit=np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisit))
               
                for i,unvis in enumerate(unvisit):
                      probabilities[i] = (pheromone[curr_point, unvis]**alpha)*((Q/distance(points[curr_point], points[unvis]))**beta)
                  
                
                probabilities/=np.sum(probabilities)
                
                next_point=np.random.choice(unvisit,p=probabilities)   
                path.append(next_point)
                path_length+=distance(points[curr_point], points[next_point])
                visited[next_point]=True
                curr_point=next_point
            paths.append(path)
            path_lengths.append(path_length)
            
                
        pheromone*=evapo_rate
            
        for path,path_length in zip(paths,path_lengths):
                for i in range(npoints-1):
                    pheromone[path[i],path[i+1]]+=Q/path_length
                pheromone[path[-1], path[0]] += Q/path_length
    
    all_paths.extend(list(zip(paths, path_lengths)))
    all_paths.sort(key=lambda x: x[1])
    best_path, best_path_length = all_paths[len(all_paths)-1]
    for i in all_paths:
        path_coordinates = [points[idx] for idx in i[0]]
    
        if(path_check(path_coordinates,obstacles,distance_factor)):
            best_path=i[0]
            best_path_length=i[1]
            break            
    ind_map = plt.imread("india3.jpg")
    fig, ax = plt.subplots()
    ax.imshow(ind_map)
    ax.scatter(points[:, 0], points[:, 1], c='r', marker='o')
      
    for i in range(npoints-1):
        ax.plot([points[best_path[i],0], points[best_path[i+1],0]],
                [points[best_path[i],1], points[best_path[i+1],1]],
                c='g', linestyle='-', linewidth=2, marker='o')

    ax.plot([points[best_path[0],0], points[best_path[-1],0]],
            [points[best_path[0],1], points[best_path[-1],1]],
            c='g', linestyle='-', linewidth=2, marker='o')
    for obstacle in obstacles:
        ax.scatter(obstacle[0], obstacle[1], c='b', marker='s', s=50)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')

    # print(all_paths)
    print(best_path)           
    plt.show()
    # plt.savefig("graph{}".format(ants)) 
    return best_path_length
    
obst=[]
def collect_obst(image, n):
    
    plt.imshow(image)
    plt.title("Click the obstacle points")
    plt.ion()
    points = []

    def on_click(event):
        nonlocal points
        if event.inaxes:
            points.append((event.xdata, event.ydata))
            plt.scatter(event.xdata, event.ydata, color='red')
            # if len(points) > 1:
            #     for i in range(len(points)-1):             
            #         plt.plot([points[i][0], points[-1][0]], [points[i][1], points[-1][1]], color='red')
            plt.draw()

        if len(points) == n:
            plt.savefig("collected_obsts.png")
            plt.ioff()
            plt.show(block=False)
            plt.close()
            global obst
            obst = points  
            print(points)

    plt.gcf().canvas.mpl_connect('button_press_event', on_click)
    plt.show(block=False)
    

collect_obst(ind_map,5)
points = data


    

[(62.07142857142844, 437.5454545454544), (170.90259740259728, 984.4220779220777), (472.909090909091, 978.9805194805193), (325.9870129870129, 581.7467532467531), (429.37662337662323, 407.61688311688295)]


In [36]:
obstacles=np.array(obst)
distance_factor=50
n_ants=13
no_iter=40
points=np.array(points)
print(acob(points,n_ants,no_iter,1,1,0.5,0.1,obstacles,distance_factor))  

[0, 9, 8, 6, 5, 4, 3, 2, 1, 7]
2067.2541060942335


In [37]:




def perpendicular_distance(point, line_start, line_end):
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end
    
    numerator = np.abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    
    distance = numerator / denominator
    return distance






def path_check(path, obstacles, distance_factor):
    for i in range(len(path) - 1):
        point1 = path[i]
        point2 = path[i + 1]
        
        for obstacle in obstacles:
            dist = perpendicular_distance(obstacle,point1, point2)
            if dist < distance_factor:
                return False  
    return True 
    

    
    
    

def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def aco1(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,obstacles):
    points=np.array(points)
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = None
    best_path_length = np.inf
    
    for iteration in range(n_iterations):
        paths = []
        path_lengths = []
        
        for ant in range(n_ants):
            visited = [False]*n_points
            current_point = np.random.randint(n_points)
            visited[current_point] = True
            path = [current_point]
            path_length = 0
            
            while False in visited:
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))
                
                for i, unvisited_point in enumerate(unvisited):
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                
                probabilities /= np.sum(probabilities)
                
                next_point = np.random.choice(unvisited, p=probabilities)
                path.append(next_point)
                path_length += distance(points[current_point], points[next_point])
                visited[next_point] = True
                current_point = next_point
            
            paths.append(path)
            path_lengths.append(path_length)
            
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length
        
        pheromone *= evaporation_rate
        
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)
    ax.scatter(points[:,0], points[:,1] ,c='r', marker='o')
    
    
    
    
    
    ind_map = plt.imread("india3.jpg")

    fig, ax = plt.subplots()
    ax.imshow(ind_map)
    ax.scatter(points[:, 0], points[:, 1], c='r', marker='o')

    
    for i in range(len(points)-1):
        ax.plot([points[best_path[i],0], points[best_path[i+1],0]],
                [points[best_path[i],1], points[best_path[i+1],1]],
    
                c='g', linestyle='-', linewidth=2, marker='o')
        
    ax.plot([points[best_path[0],0], points[best_path[-1],0]],
            [points[best_path[0],1], points[best_path[-1],1]],

            c='g', linestyle='-', linewidth=2, marker='o')
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')

    plt.show() 
    plt.show()


                  

def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def acol(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,obstacles,distance_factor):
    # print(type(points))
    points=np.array(points)
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = None
    best_path_length = np.inf
    all_paths=[]
    for iteration in range(n_iterations):
        paths = []
        path_lengths = []
        
        for ant in range(n_ants):
            visited = [False]*n_points
            current_point = np.random.randint(n_points)
            visited[current_point] = True
            path = [current_point]
            path_length = 0
            
            while False in visited:
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))
                
                for i, unvisited_point in enumerate(unvisited):
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                
                probabilities /= np.sum(probabilities)
                
                next_point = np.random.choice(unvisited, p=probabilities)
                path.append(next_point)
                path_length += distance(points[current_point], points[next_point])
                visited[next_point] = True
                current_point = next_point
            
            paths.append(path)
            path_lengths.append(path_length)
            
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length
        
        pheromone *= evaporation_rate
        all_paths.extend(list(zip(paths, path_lengths)))
        all_paths.sort(key=lambda x: x[1])
               
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
    
     
    ind_map = plt.imread("india3.jpg")

    fig, ax = plt.subplots()
    ax.imshow(ind_map)
    ax.scatter(points[:, 0], points[:, 1], c='r', marker='o')

    for obstacle in obstacles:
        ax.scatter(obstacle[0], obstacle[1], c='b', marker='s', s=10)
    for i in range(len(points)-1):
        ax.plot([points[best_path[i],0], points[best_path[i+1],0]],
                [points[best_path[i],1], points[best_path[i+1],1]],
    
                c='g', linestyle='-', linewidth=2, marker='o')
        
    ax.plot([points[best_path[0],0], points[best_path[-1],0]],
            [points[best_path[0],1], points[best_path[-1],1]],

            c='g', linestyle='-', linewidth=2, marker='o')
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')

    plt.show() 
    print(best_path)
    print(best_path_length)
    plt.show()
    
    
  
        
import math       
def perpendicular_distance(point, line_start, line_end):
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end
    print(point)
    print(line_start)
    print(line_end)
    numerator = np.abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    
    distance = numerator / denominator
    return distance


def point_on_line(obstacle,distance_factor, theta):
    x1, y1 = obstacle
    x = x1 -distance_factor * math.cos(theta)
    y = y1 - distance_factor * math.sin(theta)
    return [x, y]



def angle_between_points(point1, point2):
    x1, y1 = point1
    x2, y2 = point2

    # Calculate the angle in radians using arctangent
    theta = math.atan2(y2 - y1, x2 - x1)

    return theta



def perpendicular_point_on_line(point_a, point_b, obstacle_c):
    vector_ab = point_b - point_a
    vector_ac = obstacle_c - point_a
    scalar_projection = np.dot(vector_ac, vector_ab) / np.dot(vector_ab, vector_ab)
    point_d = point_a + scalar_projection * vector_ab
    
    return point_d

def path_check1(path, obstacles, distance_factor):
    sol=[]
    j=0
    for i in range(len(path)-1+j):
        point1 = path[i]
        point2 = path[i + 1]
        print(distance_factor)
        sol.append(point1)
        for obstacle in obstacles:
            dist = distance(obstacle,(point1+point2)/2)
            if dist < distance_factor:
                print(dist)
                ppoint = (point1+point2)/2
                theta=angle_between_points(ppoint,obstacle)
                new_p=point_on_line(obstacle,distance_factor,theta)
                new_p=np.array(new_p)
                sol.append(new_p)
                j=j+1
    
    point1 = path[len(path)-1]
    point2 = path[0] 
    ppoint = (point1+point2)/2 
    theta=angle_between_points(ppoint,obstacle)
    new_p=point_on_line(obstacle,distance_factor,theta)
    new_p=np.array(new_p)
    sol.append(new_p)    
    sol.append(path[len(path)-1])  
            
    return sol       
        
        
        
        
def acod(points, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,distance_factor,obstacles):
    print(type(points))
    n_points = len(points)
    pheromone = np.ones((n_points, n_points))
    best_path = None
    best_path_length = np.inf
    
    for iteration in range(n_iterations):
        paths = []
        path_lengths = []
        
        for ant in range(n_ants):
            visited = [False]*n_points
            current_point = np.random.randint(n_points)
            visited[current_point] = True
            path = [current_point]
            path_length = 0
            
            while False in visited:
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))
                
                for i, unvisited_point in enumerate(unvisited):
                    probabilities[i] = pheromone[current_point, unvisited_point]**alpha / distance(points[current_point], points[unvisited_point])**beta
                
                probabilities /= np.sum(probabilities)
                
                next_point = np.random.choice(unvisited, p=probabilities)
                path.append(next_point)
                path_length += distance(points[current_point], points[next_point])
                visited[next_point] = True
                current_point = next_point
            
            paths.append(path)
            path_lengths.append(path_length)
            
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length
             
        pheromone *= evaporation_rate
        
        for path, path_length in zip(paths, path_lengths):
            for i in range(n_points-1):
                pheromone[path[i], path[i+1]] += Q/path_length
            pheromone[path[-1], path[0]] += Q/path_length
    
    fig = plt.figure(figsize=(8, 6))
    ax =  fig.add_subplot(111)
    # ax.scatter(points[:,0], points[:,1], c='r', marker='o')
    path_coordinates = [points[idx] for idx in best_path]
    print(path_coordinates)
    best_path=path_check1(path_coordinates,obstacles,distance_factor)
    print(best_path)       
    # for i in range(n_points-1):
    #     ax.plot([best_path[i][0], best_path[i+1][0]],
    #             [best_path[i][1], best_path[i+1][1]],
    #             c='g', linestyle='-', linewidth=2, marker='o')
        
    # ax.plot([best_path[0][0], best_path[len(best_path)-1][0]],
    #             [best_path[0][1], best_path[len(best_path)-1][1]],
    #         c='r', linestyle='-', linewidth=2, marker='o')   
    # ax.set_xlim([-10, 10])
    # ax.set_ylim([-10, 10])
    
    # for obstacle in obstacles:
    #     ax.scatter(obstacle[0], obstacle[1], c='b', marker='s', s=10)
    # ax.set_xlabel('X Label')
    # ax.set_ylabel('Y Label')
    # plt.savefig("graph{}".format(n_ants))
    # plt.show()    
       
    acol(best_path, n_ants=11, n_iterations=50, alpha=1, beta=3, evaporation_rate=0.5, Q=1,obstacles=obstacles,distance_factor=distance_factor)





acod(points, n_ants=20, n_iterations=50, alpha=1, beta=2, evaporation_rate=0.5, Q=1,distance_factor=distance_factor,obstacles=obstacles)        
 

<class 'numpy.ndarray'>
[array([350.47402597, 987.14285714]), array([312.38311688, 900.07792208]), array([385.84415584, 881.03246753]), array([410.33116883, 788.52597403]), array([266.12987013, 783.08441558]), array([241.64285714, 679.69480519]), array([203.55194805, 576.30519481]), array([154.57792208, 475.63636364]), array([222.5974026, 353.2012987]), array([334.14935065, 138.25974026])]
50
50
50
50
50
50
50
50
50
[array([350.47402597, 987.14285714]), array([312.38311688, 900.07792208]), array([385.84415584, 881.03246753]), array([410.33116883, 788.52597403]), array([266.12987013, 783.08441558]), array([241.64285714, 679.69480519]), array([203.55194805, 576.30519481]), array([154.57792208, 475.63636364]), array([222.5974026, 353.2012987]), array([404.89988554, 451.21607239]), array([334.14935065, 138.25974026])]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1407.196957164938
